# **Recommendation Agent**

The **Recommendation Agent** is the final agent in the recommendation process. It orchestrates how the **Activity Agent**, **Usage Agent**, **Load Agent** and **Price Agent** interact in order to deliver a recommendation at a particular date for a given household (see the dedicated notebooks in order to see the definition of these agents in detail).

The recommendation agent works as follows:

1. It requests the outputs of the **Activity Agent**, **Usage Agent**, **Load Agent** and **Price Agent**:


*   The Activity Agent returns the probability that persons are present and in an "active state" in the house at each given hour of the day.
*   The Usage Agent returns the probability that a given to-be-recommended-device will be used on the next day.
*   The Load Agent returns a typical load profile for each to-be-recommended-device.
*   The Price Agent returns the day-ahead-prices for the next 48 hours.

2. It then computes the cost associated with launching the devices at each hour of the next day (based on the devices' typical load profiles and the day ahead electricity prices).

3. Finally it recommends the cheapest launching hour among the set of hours at which users are likely present and active **[Probability(Present&Active)> Threshold]**. A recommendation  for a given device is made only if the user is likely enough to use the device on the next day **[Probability(Device_Usage)> Threshold]**


In the present notebook, we will build this **Recommendation Agent** step by step. For that purpose, we will first load and preprocess the required data with the preprocessing agents. Then, we will iteratively add functions to the **Recommendation Agent class** in order to finally build a function entitled "Pipeline", which ouputs the desired recommendations.

## **1. Load and Pre-process Data**

This part's only purpose is to load the data used in the recommendation agent. This process is described in detail in the Preparation Agent.  **[You might need to adapt some parameters when applying the script to another household than household 1]**

### **1.1 Initialize and load python scripts**

In [25]:
# loading necessary libraries
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import os 

from helper_functions import Helper
from agents import Preparation_Agent
from datetime import datetime

helper = Helper()

In [26]:
DATA_PATH = '../data/'

### **1.2 Set Params**

**Note:** For the full detail of the parameters and the preprocessing agents take a look at the Preparation Agent's notebook.

In [27]:
# load household data for Household 1
household = helper.load_household(DATA_PATH, 1)

In [30]:
threshold = 0.01
active_appliances = ['Toaster', 'Tumble Dryer', 'Dishwasher', 'Washing Machine','Television', 'Microwave', 'Kettle']
shiftable_devices = ['Tumble Dryer', 'Washing Machine', 'Dishwasher']


In [31]:
#activity params
truncation_params = {
    'features': 'all', 
    'factor': 1.5, 
    'verbose': 0
}

scale_params = {
    'features': 'all', 
    'kind': 'MinMax', 
    'verbose': 0
}

aggregate_params = {
    'resample_param': '60T'
}

activity_params = {
    'active_appliances': active_appliances,
    'threshold': threshold 
}

time_params = {
    'features': ['hour', 'day_name']
}

activity_lag_params = {
    'features': ['activity'],
    'lags': [24, 48, 72]
}

activity_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'aggregate': aggregate_params,
    'activity': activity_params,
    'time': time_params,
    'activity_lag': activity_lag_params
}

#load agent
device_params = {
    'threshold': threshold
}

load_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'aggregate': aggregate_params,
    'shiftable_devices': shiftable_devices, 
    'device': device_params
}

#usage agent

device = {
    'threshold' : threshold}

aggregate_params24_H = {
    'resample_param': '24H'
}

usage_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'activity': activity_params,
    'aggregate_hour': aggregate_params,
    'aggregate_day': aggregate_params24_H,
    'time': time_params,
    'activity_lag': activity_lag_params,
    'shiftable_devices' : shiftable_devices,
    'device': device
}

In [6]:
# calling the preparation pipeline
#prep = Preparation_Agent(household)
#activity_df = prep.pipeline_activity(household, activity_pipe_params)
#load_df, _, _ = prep.pipeline_load(household, load_pipe_params)
#usage_df = prep.pipeline_usage(household, usage_pipe_params)

#load price data
#FILE_PATH = '/content/drive/MyDrive/T4_Recommendation-system-for-demand-response-and-load-shifting/02_data/' 
#price_df = helper.create_day_ahead_prices_df(DATA_PATH, 'Day-ahead Prices_201501010000-201601010000.csv')


In [5]:
#activity_df.to_pickle('../data/processed_pickle/activity_df.pkl')
#load_df,_, _ .to_pickle('../data/processed_pickle/load_df.pkl')
#usage_df.to_pickle('../data/processed_pickle/usage_df.pkl')
#price_df.to_pickle('../data/processed_pickle/price_df.pkl')

In [32]:
# Load pickle data
activity_df = pd.read_pickle('../data/processed_pickle/activity_df.pkl')
load_df = pd.read_pickle('../data/processed_pickle/load_df.pkl')
usage_df = pd.read_pickle('../data/processed_pickle/usage_df.pkl')
price_df = pd.read_pickle('../data/processed_pickle/price_df.pkl')

## **2. Constructing the Recommendation Agent**

### **2.1 Initiliaze Agent**
In a first step, the Recommendation Agent is initialized with the preprocessed data and the name of the shiftable devices (those for which we want to make predictions). All Agents are initialized accordingly.

In [33]:
from agents import Activity_Agent, Usage_Agent, Load_Agent, Price_Agent
class Recommendation_Agent:
    import pandas as pd

    def __init__(self, activity_input, usage_input, load_input, price_input, shiftable_devices):
        self.activity_input = activity_input
        self.usage_input = usage_input
        self.load_input = load_input
        self.price_input = price_input
        self.shiftable_devices = shiftable_devices

        self.Activity_Agent = Activity_Agent(activity_input)
        #create dicionnary with Usage_Agent for each device
        self.Usage_Agent = {name: Usage_Agent(usage_input , name)  for name in shiftable_devices}
        self.Load_Agent = Load_Agent(load_input)
        self.Price_Agent = Price_Agent(price_input)

In [34]:
#initialize the Recommendation Agent with the required inputs
recommend = Recommendation_Agent(activity_df, usage_df, load_df, price_df, shiftable_devices)

### **2.2 Compute Usage Cost For Every Starting Time (each of the 24 hours of the day) For A Given Device**

This function computes the cost associated with launching a to-be-recommended-device at each hour of the next day. 

#### **2.2.1 Electricity Prices For 24 hours After Hypothetical Starting Time**
First we build up a function which gives the electricity price for the 24 hours following a hypothetical starting time. 

*   The column "Price_at_H+0" gives the electricity price for the 24 hours after 00:00:00
*   The column "Price_at_H+1" gives the electricity price for the 24 hours following 01:00:00 ( "Price_at_H+0" shifted by one hour)
* The column "Price_at_H+2" gives the electricity price for the 24 hours following 02:00:00 ( "Price_at_H+0" shifted by two hours)
*....

This function first requests the **day-ahead electricity prices** for the next 48h from the Price_Agent. Then it arranges the prices as described above.


In [35]:
def electricity_prices_from_start_time(self, date):
  import pandas as pd
  prices_48 = self.Price_Agent.return_day_ahead_prices(date)
  prices_from_start_time = pd.DataFrame()

  for i in range(24):
    prices_from_start_time["Price_at_H+"+ str(i)] = prices_48.shift(-i)

  #delete last 24 hours
  prices_from_start_time = prices_from_start_time[:-24]
  return prices_from_start_time

# add to Activity agent
setattr(Recommendation_Agent, 'electricity_prices_from_start_time', electricity_prices_from_start_time)
del electricity_prices_from_start_time

In [36]:
price = recommend.electricity_prices_from_start_time("2014-08-21")
price
#H0 prices for next 24 hours start at 00:00
#H1 prices for next 24 hours start at 01:00

,Price_at_H+0,Price_at_H+1,Price_at_H+2,Price_at_H+3,Price_at_H+4,Price_at_H+5,Price_at_H+6,Price_at_H+7,Price_at_H+8,Price_at_H+9,...,Price_at_H+14,Price_at_H+15,Price_at_H+16,Price_at_H+17,Price_at_H+18,Price_at_H+19,Price_at_H+20,Price_at_H+21,Price_at_H+22,Price_at_H+23
2014-08-21 00:00:00,32.51,32.36,31.94,31.00,33.90,38.01,38.71,39.51,49.53,48.47,...,35.45,38.89,48.03,47.05,42.15,43.34,39.73,39.03,38.04,36.47
2014-08-21 01:00:00,32.36,31.94,31.00,33.90,38.01,38.71,39.51,49.53,48.47,48.47,...,38.89,48.03,47.05,42.15,43.34,39.73,39.03,38.04,36.47,35.12
2014-08-21 02:00:00,31.94,31.00,33.90,38.01,38.71,39.51,49.53,48.47,48.47,43.21,...,48.03,47.05,42.15,43.34,39.73,39.03,38.04,36.47,35.12,33.97
2014-08-21 03:00:00,31.00,33.90,38.01,38.71,39.51,49.53,48.47,48.47,43.21,36.01,...,47.05,42.15,43.34,39.73,39.03,38.04,36.47,35.12,33.97,33.52
2014-08-21 04:00:00,33.90,38.01,38.71,39.51,49.53,48.47,48.47,43.21,36.01,35.09,...,42.15,43.34,39.73,39.03,38.04,36.47,35.12,33.97,33.52,33.52
2014-08-21 05:00:00,38.01,38.71,39.51,49.53,48.47,48.47,43.21,36.01,35.09,35.45,...,43.34,39.73,39.03,38.04,36.47,35.12,33.97,33.52,33.52,34.16
2014-08-21 06:00:00,38.71,39.51,49.53,48.47,48.47,43.21,36.01,35.09,35.45,38.89,...,39.73,39.03,38.04,36.47,35.12,33.97,33.52,33.52,34.16,35.42
2014-08-21 07:00:00,39.51,49.53,48.47,48.47,43.21,36.01,35.09,35.45,38.89,48.03,...,39.03,38.04,36.47,35.12,33.97,33.52,33.52,34.16,35.42,35.56
2014-08-21 08:00:00,49.53,48.47,48.47,43.21,36.01,35.09,35.45,38.89,48.03,47.05,...,38.04,36.47,35.12,33.97,33.52,33.52,34.16,35.42,35.56,41.95
2014-08-21 09:00:00,48.47,48.47,43.21,36.01,35.09,35.45,38.89,48.03,47.05,42.15,...,36.47,35.12,33.97,33.52,33.52,34.16,35.42,35.56,41.95,49.93


#### **2.2.2 Device Launching Cost By Hour Of The Day**
We compute the cost of operating the device at every given hour by multiplying the **day ahead electricity price** with the device's **typical load profile**. The latter typical load profile is generated by the **Load Agent** (for details see the Load Agent's notebook).

As an output, we get the typical costs of operating the device for all possible 24 staring times.

**Note:** The Recommendation Agent's pipeline functions can be sped up by providing the predecessing agents' outputs directly. This Functionality will be used to evaluate the recommender systems performance.

In [41]:
def cost_by_starting_time(self, date, device, evaluation=False):
    import numpy as np
    import pandas as pd

    # get electriciy prices following every device starting hour with previously defined function
    prices = self.electricity_prices_from_start_time(date)

    # build up table with typical load profile repeated for every hour (see Load_Agent)
    if not evaluation:
        device_load = self.Load_Agent.pipeline(self.load_input, date, self.shiftable_devices).loc[device]
    else:
        # get device load for one date
        device_load = evaluation["load"][date].loc[device]

    device_load = pd.concat([device_load] * 24, axis=1)

    # multiply both tables and aggregate costs for each starting hour
    costs = np.array(prices) * np.array(device_load)
    costs = np.sum(costs, axis=0)

    # return an array of size 24 containing the total cost at each staring hour.
    return costs

setattr(Recommendation_Agent, 'cost_by_starting_time', cost_by_starting_time)
del cost_by_starting_time

As can be seen below, the output returns the cost associated with starting the "Washing machine" on each hour of the "2014-02-20".

In [3]:
recommend.cost_by_starting_time("2014-02-20", "Washing Machine")

NameError: name 'recommend' is not defined

### **2.3 Starting Time Recommendation For Each Device**

We create a function that gives a starting time recommendation for a given device.

1.   The function loads the **device usage costs associated with each starting time** (see above function).
2.   In order **not** to recommend to start the device at an hour where the person is not home or at sleep, we exclude hours that have an  **activity probability below a certain threshold**. These probabilities are computed with the **Activity Agent**.
3. In order **not** to make a recommendation when the household is unlikely to use the device on the next day anyway, we set a **device usage probability threshold** under which no recommendation is made. These usage probabilities are computed with the **Usage Agent**.


The function outputs a dictionary with the best starting time, among the hours at which activity is likely enough. The output additionaly contains **"no_recommend"** flags, in order to signal that no recommendation should be made when :
* There is no hour of the day where activity is likely enough (all hours of the day have an activity probability below the set threshold), then the **"no_recommend_flag_activity"** turns from 0 to 1.
* The device usage is unlikely (below the set threshold), then the **"no_recommend_flag_usage"** turns from 0 to 1.


In [49]:
#return cheapest launching hour from the set of hours satifsfying:  probability of activity > threshold
def recommend_by_device(self, date, device,  activity_prob_threshold, usage_prob_threshold, evaluation=False):
    import numpy as np

    split_params = {
        "train_start": "2013-11-01",
        "test_delta": {"days": 1, "seconds": -1},
        "target": "activity",
    }

    # compute costs by launching time:
    costs = self.cost_by_starting_time(date, device, evaluation=evaluation)

    # compute activity probabilities
    if not evaluation:
        activity_probs = self.Activity_Agent.pipeline(self.activity_input, date, 'logit', split_params)
    else:
        # get activity probs for date
        activity_probs = evaluation["activity"][date]

    # set values above threshold to 1. Values below to Inf
    # (vector will be multiplied by costs, so that hours of little activity likelihood get cost = Inf)
    activity_probs = np.where(activity_probs >= activity_prob_threshold, 1, float("Inf"))

    # add a flag in case all hours have likelihood smaller than threshold
    no_recommend_flag_activity = 0
    if np.min(activity_probs) == float("Inf"):
        no_recommend_flag_activity = 1

    # compute cheapest hour from likely ones
    best_hour = np.argmin(np.array(costs) * np.array(activity_probs))

    # compute likelihood of usage:
    if not evaluation:
        usage_prob = self.Usage_Agent[device].pipeline(self.usage_input, date, 'logit', split_params["train_start"])
    else:
        # get usage probs
        name = "usage_" + device.replace(" ", "_").replace("(", "").replace(")", "").lower()
        usage_prob = evaluation[name][date]

    
    no_recommend_flag_usage = 0
    if usage_prob < usage_prob_threshold:
        no_recommend_flag_usage = 1
    else:
        no_recommend_flag_usage = 0
        if usage_prob < usage_prob_threshold:
            no_recommend_flag_usage = 1

    return {
        "recommendation_date": [date],
        "device": [device],
        "best_launch_hour": [best_hour],
        "no_recommend_flag_activity": [no_recommend_flag_activity],
        "no_recommend_flag_usage": [no_recommend_flag_usage],
        "recommendation": [best_hour if (no_recommend_flag_activity == 0 and no_recommend_flag_usage == 0)else np.nan]
    }

setattr(Recommendation_Agent, 'recommend_by_device', recommend_by_device)
del recommend_by_device

In [51]:
recommendation_by_decvice_dict = recommend.recommend_by_device("2014-08-21", "Dishwasher", 0.3, 0.3, evaluation=False)

In [67]:
recommendation_by_decvice_dict

{'recommendation_date': ['2014-08-21'],
 'device': ['Dishwasher'],
 'best_launch_hour': [12],
 'no_recommend_flag_activity': [0],
 'no_recommend_flag_usage': [0],
 'recommendation': [12]}

In [78]:
def visualize_recommendation_by_device(self, dict):
    recommendation_date = str(dict['recommendation_date'][0])
    recommendation_date = datetime.strptime(recommendation_date, '%Y-%m-%d')
    recommendation_date = recommendation_date.strftime(format = "%d.%m.%Y %H:%M")
    device = dict['device'][0]
    best_launch_hour = dict['best_launch_hour'][0] 
    if (dict['no_recommend_flag_activity'][0]== 0 and dict['no_recommend_flag_usage'][0]==0) == True:
        return print('You have one recommendation for the following device: ' + device + '\nPlease use it on ' + recommendation_date[0:10] + ' at '+ recommendation_date[11:]+'.')

setattr(Recommendation_Agent, 'visualize_recommendation_by_device', visualize_recommendation_by_device)
del visualize_recommendation_by_device

In [2]:
recommend.visualize_recommendation_by_device(recommendation_by_decvice_dict)

NameError: name 'recommend' is not defined

### **2.4 Create Recommendation Function For Entire Household**
Finally, we wrap up the "recommend_by_device" function that makes a recommendation for each device, into the "pipeline" function that will make recommendations for all shiftable devices within a household.

In [103]:
def pipeline(self, date, activity_prob_threshold, usage_prob_threshold, evaluation=False, price=price):
    import pandas as pd
    
    recommendations_by_device = self.recommend_by_device(date, self.shiftable_devices[0], model_type, activity_prob_threshold,              usage_prob_threshold, evaluation=evaluation)
    recommendations_table = pd.DataFrame.from_dict(recommendations_by_device)

    for device in self.shiftable_devices[1:]:
        recommendations_by_device = self.recommend_by_device(date, device, , activity_prob_threshold, usage_prob_threshold, evaluation=evaluation)
        recommendations_table = recommendations_table.append(pd.DataFrame.from_dict(recommendations_by_device))
        #price = price
        for i in range(len(recommendations_table)):
            date_and_time = recommendations_table.recommendation_date.iloc[i] + ':' + str(recommendations_table.best_launch_hour.iloc[i])
            
            date_and_time =  datetime.strptime(date_and_time, '%Y-%m-%d:%H')

            date_and_time_show = date_and_time.strftime(format = "%d.%m.%Y %H:%M") 
            date_and_time_price = date_and_time.strftime(format = "%Y-%m-%d %H:%M:%S") 
            price = price.filter(like=date_and_time_price, axis=0)['Price_at_H+0'].iloc[0]	
           
            if (recommendations_table.no_recommend_flag_activity.iloc[i]==0 and recommendations_table.no_recommend_flag_usage.iloc[i]==0) == True:
                output = print('You have a recommendation for the following device: ' + recommendations_table.device.iloc[i]+ '\n\n Please use the device on the ' + date_and_time_show[0:10] + ' at ' + date_and_time_show[11:] + ' Uhr because it cost you only ' + str(price) + ' €.\n') 
                
                return output

setattr(Recommendation_Agent, 'pipeline', pipeline)
del pipeline

We can then generate a recommendation for a household by specifying:
1. The day to be recommended
2. The "activity_prob_threshold" (hours that have a smaller probability of household activity are not considered for recommendation)
3. The "usage_probability_threshold" (devices that have a smaller probability of usage are not considered for recommendation)

**Note**:  It remains to be investigated at which value to set these thresholds.

In [108]:
recommend.pipeline(date = "2014-08-21", model_type= "logit", activity_prob_threshold = 0.4,  usage_prob_threshold = 0.3, price=price)

You have a recommendation for the following device: Tumble Dryer

 Please use the device on the 21.08.2014 at 12:00 Uhr because it cost you only 36.01 €.



Finally, the system recommends to the user the "best_launch_hour" on the "recommendation_date" if both the "no_recommend_flag_activity" and "no_recommend_flag_usage" are at 0 for the given device. 

## **Appendix A1: Complete Recommendation Agent Class**

In [19]:
class Recommendation_Agent:
    def __init__(
        self, activity_input, usage_input, load_input, price_input, shiftable_devices
    ):
        self.activity_input = activity_input
        self.usage_input = usage_input
        self.load_input = load_input
        self.price_input = price_input
        self.shiftable_devices = shiftable_devices
        self.Activity_Agent = Activity_Agent(activity_input)
        # create dicionnary with Usage_Agent for each device
        self.Usage_Agent = {
            name: Usage_Agent(usage_input, name) for name in shiftable_devices
        }
        self.Load_Agent = Load_Agent(load_input)
        self.Price_Agent = Price_Agent(price_input)

    # calculating costs
    # -------------------------------------------------------------------------------------------
    def electricity_prices_from_start_time(self, date):
        import pandas as pd

        prices_48 = self.Price_Agent.return_day_ahead_prices(date)
        prices_from_start_time = pd.DataFrame()
        for i in range(24):
            prices_from_start_time["Price_at_H+" + str(i)] = prices_48.shift(-i)
        # delete last 24 hours
        prices_from_start_time = prices_from_start_time[:-24]
        return prices_from_start_time

    def cost_by_starting_time(self, date, device, evaluation=False):
        import numpy as np
        import pandas as pd

        # get electriciy prices following every device starting hour with previously defined function
        prices = self.electricity_prices_from_start_time(date)
        # build up table with typical load profile repeated for every hour (see Load_Agent)
        if not evaluation:
            device_load = self.Load_Agent.pipeline(
                self.load_input, date, self.shiftable_devices
            ).loc[device]
        else:
            # get device load for one date
            device_load = evaluation["load"][date].loc[device]
        device_load = pd.concat([device_load] * 24, axis=1)
        # multiply both tables and aggregate costs for each starting hour
        costs = np.array(prices) * np.array(device_load)
        costs = np.sum(costs, axis=0)
        # return an array of size 24 containing the total cost at each staring hour.
        return costs
    
    # creating recommendations
    # -------------------------------------------------------------------------------------------
    def recommend_by_device(
        self,
        date,
        device,
        model_type,
        activity_prob_threshold,
        usage_prob_threshold,
        evaluation=False,
    ):
        import numpy as np

        # add split params as input
        # IN PARTICULAR --> Specify date to start training
        split_params = {
            "train_start": "2013-11-01",
            "test_delta": {"days": 1, "seconds": -1},
            "target": "activity",
        }
        # compute costs by launching time:
        costs = self.cost_by_starting_time(date, device, evaluation=evaluation)
        # compute activity probabilities
        if not evaluation:
            activity_probs = self.Activity_Agent.pipeline(self.activity_input, date, model_type, split_params)
        else:
            # get activity probs for date
            activity_probs = evaluation["activity"][date]

        # set values above threshold to 1. Values below to Inf
        # (vector will be multiplied by costs, so that hours of little activity likelihood get cost = Inf)
        activity_probs = np.where(activity_probs >= activity_prob_threshold, 1, float("Inf"))

        # add a flag in case all hours have likelihood smaller than threshold
        no_recommend_flag_activity = 0
        if np.min(activity_probs) == float("Inf"):
            no_recommend_flag_activity = 1

        # compute cheapest hour from likely ones
        best_hour = np.argmin(np.array(costs) * np.array(activity_probs))

        # compute likelihood of usage:
        if not evaluation:
            usage_prob = self.Usage_Agent[device].pipeline(self.usage_input, date, model_type, split_params["train_start"])
        else:
            # get usage probs
            name = ("usage_"+ device.replace(" ", "_").replace("(", "").replace(")", "").lower())
            usage_prob = evaluation[name][date]

        if model_type == 'logit':
            no_recommend_flag_usage = 0
            if (usage_prob < usage_prob_threshold).bool():
            no_recommend_flag_usage = 1
        else:
            no_recommend_flag_usage = 0
            if usage_prob < usage_prob_threshold:
                no_recommend_flag_usage = 1

        return {
            "recommendation_date": [date],
            "device": [device],
            "best_launch_hour": [best_hour],
            "no_recommend_flag_activity": [no_recommend_flag_activity],
            "no_recommend_flag_usage": [no_recommend_flag_usage],
            "recommendation": [
                best_hour
                if (no_recommend_flag_activity == 0 and no_recommend_flag_usage == 0)
                else np.nan
            ],
        }

    # vizualizing the recommendations
    # -------------------------------------------------------------------------------------------
    def recommendations_on_date_range(
        self, date_range, activity_prob_threshold=0.6, usage_prob_threshold=0.5
    ):
        import pandas as pd

        recommendations = []
        for date in date_range:
            recommendations.append(self.pipeline(date, activity_prob_threshold, usage_prob_threshold))
            output = pd.concat(recommendations)
        return output

    def visualize_recommendations_on_date_range(self, recs):
        import plotly.express as px
        import plotly.graph_objects as go

        fig = go.Figure()

        for device in recs["device"].unique():
            plot_device = recs[recs["device"] == device]
            fig.add_trace(
                go.Scatter(
                    x=plot_device["recommendation_date"],
                    y=plot_device["recommendation"],
                    mode="lines",
                    name=device,
                )
            )
        fig.show()

    def histogram_recommendation_hour(self, recs):
        import seaborn as sns

        ax = sns.displot(recs, x="recommendation", binwidth=1)
        ax.set(xlabel="Hour of Recommendation", ylabel="counts")
    
    # pipeline function: create recommendations
    # -------------------------------------------------------------------------------------------
   def pipeline(self, date, model_type, activity_prob_threshold, usage_prob_threshold, evaluation=False, price=price):
    import pandas as pd
    
    recommendations_by_device = self.recommend_by_device(date, self.shiftable_devices[0], model_type, activity_prob_threshold,              usage_prob_threshold, evaluation=evaluation)
    recommendations_table = pd.DataFrame.from_dict(recommendations_by_device)

    for device in self.shiftable_devices[1:]:
        recommendations_by_device = self.recommend_by_device(date, device, model_type, activity_prob_threshold, usage_prob_threshold, evaluation=evaluation)
        recommendations_table = recommendations_table.append(pd.DataFrame.from_dict(recommendations_by_device))
        #price = price
        for i in range(len(recommendations_table)):
            date_and_time = recommendations_table.recommendation_date.iloc[i] + ':' + str(recommendations_table.best_launch_hour.iloc[i])
            
            date_and_time =  datetime.strptime(date_and_time, '%Y-%m-%d:%H')

            date_and_time_show = date_and_time.strftime(format = "%d.%m.%Y %H:%M") 
            date_and_time_price = date_and_time.strftime(format = "%Y-%m-%d %H:%M:%S") 
            price = price.filter(like=date_and_time_price, axis=0)['Price_at_H+0'].iloc[0]	
            output = print('You have a recommendation for the following device: ' + recommendations_table.device.iloc[i]+ '\n\n Please use the device on the ' + date_and_time_show[0:10] + ' at ' + date_and_time_show[11:] + ' Uhr because it cost you only ' + str(price) + ' €.\n') 
            if (recommendations_table.no_recommend_flag_activity.iloc[i]==0 and recommendations_table.no_recommend_flag_usage.iloc[i]==0) == True:
                
                return output